# This file generates the cleaned grocery store data for analysis

In [1]:
import pandas as pd
import numpy as np
import folium
import geopandas as gpd
import matplotlib.pyplot as plt
from datetime import datetime
from dateutil.relativedelta import relativedelta
from shapely.geometry import Point, Polygon
import math
import geopy
import geopy.distance
from geopy.distance import geodesic, great_circle

In [2]:
num_months = 3
radius = 0.4 # in kilometers

In [3]:
comm_areas = gpd.read_file("Comm_Areas.geojson")
comm_areas = comm_areas.to_crs(epsg=4626)
only_violent_crimes = True

In [4]:
bl = pd.read_csv("Business_Licenses.csv")
bl = bl[["LATITUDE", "LONGITUDE", "LICENSE ID", "LICENSE DESCRIPTION", "BUSINESS ACTIVITY", "APPLICATION TYPE", "DATE ISSUED"]]

C:\Users\patri\AppData\Local\Temp\ipykernel_14148\283549513.py:1: DtypeWarning: Columns (9) have mixed types. Specify dtype option on import or set low_memory=False.
  bl = pd.read_csv("Business_Licenses.csv")


In [5]:
groceries = pd.read_csv("Grocery_Stores_-_2013.csv")

In [6]:
grocery_issuances = bl[bl['LICENSE ID'].isin(groceries["LICENSE ID"]) & bl['APPLICATION TYPE'].str.contains('ISSUE', case=False)]
grocery_issuances = grocery_issuances.dropna()
grocery_issuances["DATE ISSUED"] =  pd.to_datetime(grocery_issuances["DATE ISSUED"], format="%m/%d/%Y")
grocery_issuances = grocery_issuances[grocery_issuances['DATE ISSUED'] < datetime.now() + relativedelta(months=-9)]
grocery_issuances = grocery_issuances.reset_index(drop=True)

In [7]:
grocery_issuances.count()

LATITUDE               198
LONGITUDE              198
LICENSE ID             198
LICENSE DESCRIPTION    198
BUSINESS ACTIVITY      198
APPLICATION TYPE       198
DATE ISSUED            198
dtype: int64

In [8]:
crimes = pd.read_csv("Crimes_2001_to_Present.csv")
crimes = crimes[["Primary Type", "Date", "Latitude", "Longitude", "Community Area"]]
crimes['Date']= pd.to_datetime(crimes['Date'])
crimes = crimes.dropna()
if only_violent_crimes:
    crimes = crimes[crimes['Primary Type'].isin(['BATTERY', 'THEFT', 'ASSAULT', 'ROBBERY',
         'CRIM SEXUAL ASSAULT', 'ARSON',  'KIDNAPPING','CRIMINAL SEXUAL ASSAULT','HOMICIDE'])]
crimes

,Primary Type,Date,Latitude,Longitude,Community Area
0,BATTERY,2015-09-05 13:30:00,41.815117,-87.670000,61.0
1,THEFT,2015-09-04 11:30:00,41.895080,-87.765400,25.0
4,ASSAULT,2015-09-05 13:00:00,41.881903,-87.755121,25.0
7,THEFT,2015-09-05 13:00:00,41.851989,-87.689219,31.0
8,ROBBERY,2015-09-05 11:30:00,41.882814,-87.704326,27.0
...,...,...,...,...,...
7663768,THEFT,2022-08-02 15:08:00,42.014840,-87.683976,2.0
7663776,ROBBERY,2022-09-02 12:00:00,41.867429,-87.626343,32.0
7663777,HOMICIDE,2022-09-27 22:44:00,41.838383,-87.620519,35.0
7663783,THEFT,2022-09-16 13:48:00,41.928077,-87.785606,19.0


In [9]:
def get_rectangle_points(coordinates, bearing, width, height):
    start = geopy.Point(coordinates)
    hypotenuse = np.hypot(width, height)
    d = geopy.distance.distance(kilometers=hypotenuse/2)

    opposite_angle = np.degrees(np.arctan(width/height))
    northeast_angle = 0 - opposite_angle
    southwest_angle = 180  - opposite_angle
    northwest_angle = opposite_angle
    southeast_angle = 180 + opposite_angle

    points = []
    for angle in [northeast_angle, northwest_angle, southwest_angle, southeast_angle]:
        point = d.destination(point=start, bearing=angle+bearing)
        coords = (point.latitude, point.longitude)
        #coords = (point.longitude, point.latitude)
        points.append(coords)

    return points

# for each grocery license create a 1km by 1km square around to associate with crimes
grocery_issuances['geometry'] = grocery_issuances.apply(lambda row: Polygon(get_rectangle_points((row['LATITUDE'], row['LONGITUDE']), 0, 1, 1)), axis=1)
# for each crime create a point in series
crimes['geometry'] = crimes.apply(lambda r: Point(r['Latitude'], r['Longitude']), axis=1)

C:\Users\patri\AppData\Local\Programs\Python\Python310\lib\site-packages\pandas\core\dtypes\cast.py:128: ShapelyDeprecationWarning: The array interface is deprecated and will no longer work in Shapely 2.0. Convert the '.coords' to a numpy array instead.
  arr = construct_1d_object_array_from_listlike(values)


In [10]:
# Assign the community area number to each issuance
grocery_issuances["comm_id"] = float("NaN")
comm_idx = grocery_issuances.columns.get_loc("comm_id")
for i in range(len(grocery_issuances)):
    lat = grocery_issuances['LATITUDE'].iloc[i]
    long = grocery_issuances['LONGITUDE'].iloc[i]
    if np.isnan(lat) or np.isnan(long):
        continue
    comm_ID = -1
    for _, r in comm_areas.iterrows():
        sim_geo = gpd.GeoSeries(r['geometry']).simplify(tolerance=0.001)
        if sim_geo.intersects(Point(long, lat)).any():
            comm_ID = r['area_num_1']
            grocery_issuances.iat[i, comm_idx] = comm_ID

In [11]:
grocery_issuances= grocery_issuances.dropna()

In [12]:
# for each grocery issuance generate a control unit in same area that does not have a grocery store within n months (prior or after)
control_positions = []
for _, r in grocery_issuances.iterrows():
    comm_id = r['comm_id']
    comm_geom = comm_areas[comm_areas['area_num_1'] == comm_id]['geometry']
    grocery_in_date_range = grocery_issuances[(grocery_issuances['DATE ISSUED'] > r['DATE ISSUED'] + relativedelta(months=-num_months)) 
                                            & (grocery_issuances['DATE ISSUED'] < r['DATE ISSUED'] + relativedelta(months=+num_months))]
    control_unit = None
    num_attempts = 0
    while control_unit == None:
        if num_attempts > 5:
            break
        num_attempts += 1
        # find the bounds of your geodataframe
        x_min, y_min, x_max, y_max = comm_geom.total_bounds
        # set sample size
        n = 5
        # generate random data within the bounds
        try:
            x = np.random.uniform(x_min, x_max, n)
            y = np.random.uniform(y_min, y_max, n)
        except:
            print("Exception for row: ", r)
            print(x_min, y_min, x_max, y_max)
        # convert them to a points GeoSeries
        gdf_points = gpd.GeoSeries(gpd.points_from_xy(x, y))
        # only keep those points within polygons
        gdf_points = gdf_points[gdf_points.within(comm_geom.unary_union)]
        
        # generate a random point in the same community area
        # if the point is not +-1km of grocery store created +-3months then add it to control units
        is_valid = True
        for point in gdf_points:
            for _, s in grocery_issuances.iterrows():
                if great_circle((point.y, point.x), (s["LATITUDE"], s["LONGITUDE"])).km < radius:
                    is_valid = False
                    break
            if is_valid:
                control_unit = point
                break
    control_positions.append(control_unit)

In [13]:
index_to_drop = [i for i in range(len(control_positions)) if control_positions[i] is None]
grocery_issuances = grocery_issuances.drop(grocery_issuances.index[index_to_drop])
grocery_issuances = grocery_issuances.reset_index(drop=True)
control_positions = [x for x in control_positions if x is not None]

In [14]:
control_units = []
for i in range(len(grocery_issuances)):
    geometry = Polygon(get_rectangle_points((control_positions[i].y, control_positions[i].x), 0, 1, 1))
    date = grocery_issuances['DATE ISSUED'].iloc[i]
    comm_id = grocery_issuances['comm_id'].iloc[i]
    control_units.append((geometry, date, comm_id, control_positions[i].y, control_positions[i].x))
control_units = pd.DataFrame(control_units, columns = ['geometry', 'DATE ISSUED', 'comm_id', "LATITUDE", "LONGITUDE"])

In [15]:
chi_map = folium.Map(location=[41.895140898, -87.624255632],
                        zoom_start=14,
                        tiles="cartodbpositron")
for _, r in comm_areas.iterrows():
    sim_geo = gpd.GeoSeries(r['geometry']).simplify(tolerance=0.001)
    geo_j = sim_geo.to_json()
    geo_j = folium.GeoJson(data=geo_j,
                           style_function=lambda x: {'fillColor': 'orange'})
    popup_text = """COMMUNITY ID: {}<br>""".format(r['community'], r['area_num_1'])

    folium.Popup(popup_text).add_to(geo_j)
    geo_j.add_to(chi_map)
    
# plot crimes 
for _, r in crimes.head(1000).iterrows():
    lat = r['Latitude']
    long = r['Longitude']
    if np.isnan(lat) or np.isnan(long):
        continue
    comm_ID = r['Community Area']
    color = "#0000FF"
    
    popup_text = """CRIME<br>
                    Latitude : {}<br>
                    Longitude : {}<br>
                    CommID : {}<br>""".format(lat, long, comm_ID)
    folium.CircleMarker(location = [lat, long], popup= popup_text,radius=radius, color = color, fill = True).add_to(chi_map)

draw_num = int(len(grocery_issuances)/5)
for i in range(draw_num):
    lat = grocery_issuances['LATITUDE'].iloc[i]
    long = grocery_issuances['LONGITUDE'].iloc[i]
    comm_ID = grocery_issuances['comm_id'].iloc[i]
    if np.isnan(lat) or np.isnan(long):
        continue
    date_issued = grocery_issuances['DATE ISSUED'].iloc[i]        
    popup_text = """LICENSE ISSUANCE<br>
                    Latitude : {}<br>
                    Longitude : {}<br>
                    CommID : {}<br>
                    Date Issued : {}<br>""".format(lat, long, comm_ID, date_issued)
    folium.CircleMarker(location = [lat, long], popup= popup_text, radius=radius, color = "#FF4500", fill = True).add_to(chi_map)
    folium.Polygon(get_rectangle_points((lat, long), 0, 1, 1), color='orange', fill=False).add_to(chi_map)

for i in range(draw_num):
    lat = control_positions[i].y
    long = control_positions[i].x
    folium.CircleMarker(location = [lat, long], popup='CONTROL UNIT', radius=radius, color = "yellow", fill = True).add_to(chi_map)
    folium.Polygon(get_rectangle_points((lat, long), 0, 1, 1), color='yellow', fill=False).add_to(chi_map)
chi_map

In [16]:
# count the number of crimes in each rectangle/circle around a grocery issuance within 3 months after the issue date
def get_num_crimes(r):
    issue_date = r['DATE ISSUED']
    center_pos = (r['LATITUDE'], r['LONGITUDE'])
    if num_months > 0:
        crimes_in_date_range = crimes[(crimes['Date']>issue_date) & (crimes['Date']<issue_date + relativedelta(months=+num_months))]  
    else:
        crimes_in_date_range = crimes[(crimes['Date']<issue_date) & (crimes['Date']>issue_date + relativedelta(months=+num_months))]  
    lats = np.array(crimes_in_date_range['Latitude'])
    longs = np.array(crimes_in_date_range['Longitude'])
    within_radius = [great_circle((lats[i], longs[i]), center_pos).km < radius for i in range(len(lats))]
    # # Code to use rectangles instead
    # crime_points = gpd.GeoSeries(crimes_in_date_range['geometry'])
    # return crime_points.within(r['geometry']).sum()
    # print(np.array(within_radius).sum())
    return np.array(within_radius).sum()
# Set new crime Rate
grocery_issuances['num crimes'] = grocery_issuances.apply(get_num_crimes, axis=1)
print("reach1")
control_units['num crimes'] = control_units.apply(get_num_crimes, axis=1)

# Set old crime rate
num_months = -num_months
control_units['num old crimes'] = control_units.apply(get_num_crimes, axis=1)
print("reach2")
grocery_issuances['num old crimes'] = grocery_issuances.apply(get_num_crimes, axis=1)

# Set change in crime
control_units['CHANGE IN CRIMES'] = control_units.apply(lambda r: r['num crimes'] - r['num old crimes'], axis=1)
grocery_issuances['CHANGE IN CRIMES'] = grocery_issuances.apply(lambda r: r['num crimes'] - r['num old crimes'], axis=1)

reach1
reach2


In [17]:
# generate confounders
socioeconomic_stats = pd.read_csv("socioeconomic.csv").dropna()
socioeconomic_stats['Community Area Number'] = socioeconomic_stats['Community Area Number'].astype('int')
socioeconomic_stats = socioeconomic_stats.set_index('Community Area Number')
health_stats = pd.read_csv("Public-Health-Stats.csv")
health_stats['Community Area'] = health_stats['Community Area'].astype('int')
health_stats = health_stats.set_index("Community Area")
confounders = pd.concat([socioeconomic_stats, health_stats], axis=1)
confounders = confounders.drop(["COMMUNITY AREA NAME", "Community Area Name", "Assault (Homicide)", "Firearm-related"], axis=1)

In [18]:
# make data that will be used in ATE estimation
a = [1 for i in range(len(grocery_issuances))]
a += [0 for i in range(len(control_units))]
x = [confounders.loc[int(grocery_issuances["comm_id"].iloc[i])].values for i in range(len(grocery_issuances))]
x += [confounders.loc[int(control_units["comm_id"].iloc[i])].values for i in range(len(control_units))]
# lol y also includes some confounders now. Don't worry they are seperated in analysis.
y = pd.concat([grocery_issuances[['CHANGE IN CRIMES', 'num old crimes', 'num crimes', "DATE ISSUED"]], control_units[['CHANGE IN CRIMES', 'num old crimes', 'num crimes', "DATE ISSUED"]]])

In [19]:
x = np.array(x).reshape(2*len(grocery_issuances),32)
y = y.reset_index(drop=True)
a = pd.DataFrame(a, columns = ['IS TREATED'])
x = pd.DataFrame(x, columns = ['PERCENT OF HOUSING CROWDED', 'PERCENT HOUSEHOLDS BELOW POVERTY',
       'PERCENT AGED 16+ UNEMPLOYED', 'PERCENT AGED 25+ WITHOUT HIGH SCHOOL DIPLOMA',
       'PERCENT AGED UNDER 18 OR OVER 64', 'PER CAPITA INCOME ',
       'HARDSHIP INDEX', 'Birth Rate', 'General Fertility Rate',
       'Low Birth Weight', 'Prenatal Care Beginning in First Trimester',
       'Preterm Births', 'Teen Birth Rate', 'Breast cancer in females',
       'Cancer (All Sites)', 'Colorectal Cancer', 'Diabetes-related',
       'Infant Mortality Rate', 'Lung Cancer', 'Prostate Cancer in Males',
       'Stroke (Cerebrovascular Disease)',
       'Childhood Blood Lead Level Screening', 'Childhood Lead Poisoning',
       'Gonorrhea in Females', 'Gonorrhea in Males', 'Tuberculosis',
       'Below Poverty Level', 'Crowded Housing', 'Dependency',
       'No High School Diploma', 'Per Capita Income', 'Unemployment'])

In [20]:
data = pd.concat([a,x,y], axis=1)

In [21]:
data

,IS TREATED,PERCENT OF HOUSING CROWDED,PERCENT HOUSEHOLDS BELOW POVERTY,PERCENT AGED 16+ UNEMPLOYED,PERCENT AGED 25+ WITHOUT HIGH SCHOOL DIPLOMA,PERCENT AGED UNDER 18 OR OVER 64,PER CAPITA INCOME,HARDSHIP INDEX,Birth Rate,General Fertility Rate,...,Below Poverty Level,Crowded Housing,Dependency,No High School Diploma,Per Capita Income,Unemployment,CHANGE IN CRIMES,num old crimes,num crimes,DATE ISSUED
0,1,14.8,33.9,17.3,35.4,38.0,13781,85.0,19.2,80.7,...,32.6,11.2,38.3,36.8,13391,12.3,16,39,55,2006-06-20
1,1,10.8,18.7,14.6,37.3,37.3,15461,70.0,20.0,88.6,...,18.6,10.0,36.9,37.0,15246,11.5,-7,29,22,2006-08-14
2,1,6.0,15.3,9.2,24.7,31.0,20039,42.0,18.5,77.7,...,14.6,5.8,30.4,25.7,20489,9.3,-15,33,18,2006-07-06
3,1,6.3,28.6,22.6,24.4,37.9,15957,73.0,18.0,80.1,...,27.0,5.7,39.0,25.0,15920,21.0,5,35,40,2006-07-05
4,1,4.0,27.6,28.3,18.5,41.9,15528,74.0,15.1,70.5,...,24.5,4.1,42.1,19.5,16022,24.2,-23,63,40,2005-10-25
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
371,0,4.8,34.4,35.9,26.3,40.7,11317,89.0,20.3,93.3,...,32.3,6.9,40.9,30.3,10559,34.7,-4,44,40,2010-11-30
372,0,6.3,28.6,22.6,24.4,37.9,15957,73.0,18.0,80.1,...,27.0,5.7,39.0,25.0,15920,21.0,1,16,17,2003-09-09
373,0,3.8,46.6,28.0,28.5,42.5,11888,94.0,20.0,85.8,...,42.2,4.8,43.4,29.4,11993,21.3,8,6,14,2003-07-07
374,0,6.3,28.6,22.6,24.4,37.9,15957,73.0,18.0,80.1,...,27.0,5.7,39.0,25.0,15920,21.0,19,46,65,2002-07-02


In [22]:
data.to_csv('cleaned_grocery_data_violent_0.4km-circles.csv', sep='\t')

In [23]:
# check how many issuances are overlapping region and within the date range
count = 0
for _, r in grocery_issuances.iterrows():
    issue_date = r['DATE ISSUED']
    center_pos = (r['LATITUDE'], r['LONGITUDE'])
    lic_id = r['LICENSE ID']
    other_in_date_range = grocery_issuances[(grocery_issuances['LICENSE ID']>lic_id)
                                           & (grocery_issuances['DATE ISSUED']>issue_date + relativedelta(months=-num_months))
                                           & (grocery_issuances['DATE ISSUED']<issue_date + relativedelta(months=+num_months))]  
    lats = np.array(other_in_date_range['LATITUDE'])
    longs = np.array(other_in_date_range['LONGITUDE'])
    within_radius = [great_circle((lats[i], longs[i]), center_pos).km < radius for i in range(len(lats))]
    count += np.array(within_radius).any()
print(f"There are {count} overlapping distance and date range grocery stores")
count = 0
for _, r in control_units.iterrows():
    issue_date = r['DATE ISSUED']
    center_pos = (r['LATITUDE'], r['LONGITUDE'])
    grocery_in_date_range = grocery_issuances[(grocery_issuances['DATE ISSUED']>issue_date + relativedelta(months=-num_months))
                                           & (grocery_issuances['DATE ISSUED']<issue_date + relativedelta(months=+num_months))]  
    lats = np.array(grocery_in_date_range['LATITUDE'])
    longs = np.array(grocery_in_date_range['LONGITUDE'])
    # print(grocery_in_date_range.count())
    within_radius = [great_circle((lats[i], longs[i]), center_pos).km < radius for i in range(len(lats))]
    count += np.array(within_radius).any()
# this value should be 0 by construction
print(f"There are {count} grocery stores nearby control units and in date range ")

There are 0 overlapping distance and date range grocery stores
There are 0 grocery stores nearby control units and in date range 
